In [17]:
import zipfile
import pathlib
import string
from collections import Counter
from typing import List

import emojis
import networkx as nx

In [53]:
%load_ext autoreload

%autoreload 2

from utils import read_text, preprocess_text
from language_model import count_ngram, NGramStat
from encoder import beam_search, build_graph, decode_ngrams, encode_text, decode_text

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [54]:
path_to_zip = pathlib.Path("data", "corpora.zip")

In [55]:
texts = read_text(path_to_zip, ignore_files=["WarAndPeaceEng.txt"])

In [56]:
n_gram = 1

In [57]:
source_freq = Counter()
for name in texts:
    if name == "AnnaKarenina.txt":
        big_text = texts[name]
    else:
        source_freq.update(count_ngram(texts[name], n_gram=n_gram))

In [58]:
unigram_stat = NGramStat(source_freq)

In [59]:
vocab = set(unigram_stat._ngram_proba.keys())

# Частотное декодирование на основе частот отдельных симовлов

In [60]:
test_text = "рассказ «Тапер» впервые появился на страницах декабрьского выпуска газеты «Одесские новости» в 1900 году. В его основу легла реальная история, которую Александр Куприн услышал в Москве. Главный герой рассказа — бедный талантливый музыкант, который по чистой случайности оказывается приглашенным играть на рождественском вечере в богатом доме. Здесь его игрой заинтересовался один из гостей — знаменитый композитор Антон Рубинштейн, который впоследствии стал его учителем."
test_text = big_text[:2000]

In [61]:
encoded_text, original_processed = encode_text(test_text, vocab, n_gram=1)

Encoded text length: 1999


In [62]:
encoded_text

'💐🎑🎑💐🎴🌙💐🆗🌊🎑🐻🎑💐🎴🈸👪🐻🎑🎴🐻😨🎴🙀💐🈁🌠🌱🎴😨🎑💐🈁🌊🎑🐻🆎🌠🌱🎴🆗🈸🈁💐🎑🈸📁🎴❌🔎📁💐🎴🆎🈸❌🙀🆎🈸🍆🈸🎴🎑💐🍐🐻🎑💐🌊🆎🙀🈺🎴🙀🆎💐📁🔴🌊🌟🎴💐🔰🈸🆗🐻😨🈁🈸🈁🎴🔰🆗💐😨🈸🌟🎴📁🙀🌊🎴🙀🍐💐🙀🆎❌🐻📁🌠🌊🎴🙀🌊🈁🔎🐻🎴💑🈸🌱🈸📻🐻🎴👪🆗🔨🍆🎴🎑💐🎴👪🆗🔨🍆💐🎴🌙💐📻👪💐🈺🎴🎑🌊🙀🍐💐🙀🆎❌🐻📁💐🈺🎴🙀🌊🈁🔎🈺🎴🎑🌊🙀🍐💐🙀🆎❌🐻📁💐🎴💑🈸🙀📁🈸🌊🈁🔨🎴😅🆎🈸🎴🌙🎑🐻🍆💐🎴🈸🎴📁🌊🍐🎑🌠🌱🎴📩🌊🎑🎑🈸🙀🆎🈺🌱🎴🈸🎴❌🆖👉📁🐻🎴🈸🎴📁🌊🆗🌊🎴🈸🎴🙀🌊🈁🔎🌊🎴🈸🎴🍐🌊❌🈸📁🌊🍐🌊🙀🌙🈸🈁🎴👪🈸🙀🆎🈸🐻🎑🙀🆎📁🌊🎴❌🌊📁🎴🆎🈸❌🙀🆎🈸🌟🆗🈸🈁💐🎑🎴🔴🐻🆗🈸🌙🈸🍆🈸🎴👪🌠🌱💐🎑🐻🈺🎴🍐💐🙀🆎🔎🎴💑🌊🆗📁💐🈺🎴❌🌊📁🎴🆎🈸❌🙀🆎🈸🌟🎴💐🎑🎑💐🎴🌙💐🆗🌊🎑🐻🎑💐🎴🆗🈸🈁💐🎑🎴🔴🐻🆗🈸🌙🈸🍆🈸🎴👪🌠🌱💐🎑🐻🈺🎴💐🎑🎑💐🎴🌙💐🆗🌊🎑🐻🎑💐🎴💑🈸🆗💐😨🐻❌💐🎴🙀🈸📁🆗🌊🈁🌊🎑🎑🐻🌙🈸📁🎴📁🙀🌊👪🎑🌊📁🎑🈸🙀🆎🔎🆖🎴🙀🈸👪🌊🆗📻💐🎑🐻🈺🎴🎑🌊🈸👉🌠🍐💐🌟🎑💐🈺🎴🙀📁🈸👉🈸👪💐🎴🆗💐🙀🌙🈸📁💐🎑🎑🈸🙀🆎🔎🎴💑🈸📁🌊🙀🆎📁🈸📁💐🎑🐻🈺🎴🔨👪🐻📁🐻🆎🌊❌🔎🎑🈸🎴🙀🈸🍐🌊🆎💐❌🐻🙀🔎🎴📁🎴😅🆎🈸🈁🎴🆗🈸🈁💐🎑🌊🎴🙀🎴📩🌊❌🔎🎑🈸🙀🆎🔎🆖🎴🌱🔨👪🈸📻🌊🙀🆎📁🌊🎑🎑🈸🍆🈸🎴📁😨🍆❌🈺👪💐🎴💐📁🆎🈸🆗💐🎴🎑💐🎴📻🐻😨🎑🔎🎴🈸🎑🎴📁🌠🙀🆎🔨💑💐❌🎴😨👪🌊🙀🔎🎴🌙💐🌙🎴🌱🔨👪🈸📻🎑🐻🌙🎴🐻🎴🈁🌠🙀❌🐻🆎🌊❌🔎🎴🐻🎴🎑💐😨🎑💐🍐🌊🎑🐻🌊🎴🐻🙀🌙🔨🙀🙀🆎📁💐🎴📁🐻👪🌊❌🎴🎑🌊🎴📁🎴🆎🈸🈁🎴🍐🆎🈸👉🌠🎴🎑🌊🈸🙀💑🈸🆗🐻🈁🈸🎴🆗💐😨🆗🌊🔴🐻🆎🔎🎴📁🈸💑🆗🈸🙀🎴💐🎴📁🎴🆎🈸🈁🎴🍐🆎🈸👉🌠🎴😨💐🙀🆎💐📁🐻🆎🔎🎴❌🆖👉🐻🆎🔎🎴📻🐻😨🎑🔎🎴📁🎴👉🌊🙀🍐🐻🙀❌🌊🎑🎑🌠🌱🎴🎑🐻🌙🈸🍆👪💐🎴🎑🌊🎴🐻🙀🆎🈸🚤🐻🈁🌠🌱🎴📁🙀🌊🌱🎴🌊🌊🎴💑🆗🈸🈺📁❌🌊🎑🐻🈺🌱🎴📁🎴🌊🎴🍆🈸👪🌠🎴🈸👪🐻🎑🎴🈁💐🙀🆎🐻🆎🌠🌟🎴💑🐻🙀💐🆎🌊❌🔎🎴💑🈸📁🐻👪🐻🈁🈸🈁🔨🎴🍆🈸🎑🍐💐🆗🈸📁🎴🙀🌙💐😨💐❌🎴👪🈸🙀🆎🈸🌊📁🙀🌙🈸🈁🔨🎴😅🆎🈸🎴📁🌊🚤🔎🎴🎑🌊🙀❌🌠🌱💐🎑🎑💐🈺🎴😅🆎🈸🎴📁🌊🚤🔎🎴💑🌊🆗📁💐🈺🎴🌙🆎🈸🎴🔨🎴🎑💐🙀🎴🐻😨🎴💑🐻🙀💐🆎🌊❌🌊🌟🎴🈁🈸📻🌊🆎🎴💑🈸🆗💐📁🎑🈺🆎🔎🙀🈺🎴🙀🎴😅🆎🐻🈁🎴💐🎴📁🎴🌊📁🆗🈸💑🌊🎴🌙🆎🈸🎴💑🆗🌊👪🙀🆎💐📁🐻🆎🎴🌱🈸🆎🔎🎴🍐🆎🈸🎑🐻👉🔨👪🔎🎴💑🈸👪🈸👉🎑🈸🌊🎴🔰🈁🎴👪🈸🙀🆎🈸🌊📁🙀🌙🐻🌟🎴🎑💐🌱🈸👪🐻❌🎴

In [65]:
for serach_res in decode_text(encoded_text, unigram_stat, n_gram, beam_search_width=10):
    print(serach_res.phrase)

Beam search:  77%|███████▋  | 1534/1998 [00:00<00:00, 15188.36it/s]Start form: (0, 'и')
иааи диреанаи омна нз тикяй заикеансяй рокиаол вгли совтсопо аибнаиесты тсилхеч ифорнзкок фризоч лте тбитсвнляе текгн ьойошн мруп аи мрупи дишмиы аетбитсвнлиы текгы аетбитсвнли ьотлоеку юсо данпи о лебаяй эеааотсый о вцжлн о лере о текге о беволебетдок мотсонатсле вел совтсочрокиа хнродопо мяйианы битсг ьерлиы вел совтсоч иааи диреанаи рокиа хнродопо мяйианы иааи диреанаи ьоризнви толрекеаандол лтемаелаотсгц томершианы аеожябичаиы тложоми ритдолиааотсг ьолетслолианы умнлнсевгао тобесивнтг л юсок рокиае т эевгаотсгц йумошетслеааопо лзпвыми илсори аи шнзаг оа лятсуьив зметг дид йумошанд н кятвнсевг н аизаибеане нтдуттсли лнмев ае л сок бсожя аеотьорнко ризрехнсг лоьрот и л сок бсожя зитсилнсг вцжнсг шнзаг л жетбнтвеааяй андопми ае нтсощнкяй лтей ее ьроылвеаный л е помя омна китснсяч ьнтисевг ьолнмнкоку поабирол тдизив мотсоелтдоку юсо лещг аетвяйиааиы юсо лещг ьерлиы дсо у аит нз ьнтисевеч кошес ьорил

In [66]:
original_processed

'анна каренина один из самых знаменитых романов льва толстого начинается ставшей афоризмом фразой все счастливые семьи похожи друг на друга каждая несчастливая семья несчастлива посвоему это книга о вечных ценностях о любви о вере о семье о человеческом достоинстве лев толстойроман широкого дыхания часть первая лев толстой анна каренина роман широкого дыхания анна каренина поразила современников вседневностью содержания необычайная свобода раскованность повествования удивительно сочетались в этом романе с цельностью художественного взгляда автора на жизнь он выступал здесь как художник и мыслитель и назначение искусства видел не в том чтобы неоспоримо разрешить вопрос а в том чтобы заставить любить жизнь в бесчисленных никогда не истощимых всех ее проявлениях в е годы один маститый писатель повидимому гончаров сказал достоевскому это вещь неслыханная это вещь первая кто у нас из писателей может поравняться с этим а в европе кто представит хоть чтонибудь подобное фм достоевский находил 

Beam search: 1999it [00:14, 15188.36it/s]                          

# Частотное декодирование на основе частот биграмм

In [67]:
n_gram = 2
source_freq = Counter()
for name in texts:
    if name == "AnnaKarenina.txt":
        big_text = texts[name]
    else:
        source_freq.update(count_ngram(texts[name], n_gram=n_gram))

In [68]:
bigram_stat = NGramStat(source_freq)

In [69]:
encoded_text, original_processed = encode_text(test_text, vocab, n_gram=n_gram)

Encoded text length: 1999


In [70]:
encoded_text

'💐🎑🎑💐🎴🌙💐🆗🌊🎑🐻🎑💐🎴🈸👪🐻🎑🎴🐻😨🎴🙀💐🈁🌠🌱🎴😨🎑💐🈁🌊🎑🐻🆎🌠🌱🎴🆗🈸🈁💐🎑🈸📁🎴❌🔎📁💐🎴🆎🈸❌🙀🆎🈸🍆🈸🎴🎑💐🍐🐻🎑💐🌊🆎🙀🈺🎴🙀🆎💐📁🔴🌊🌟🎴💐🔰🈸🆗🐻😨🈁🈸🈁🎴🔰🆗💐😨🈸🌟🎴📁🙀🌊🎴🙀🍐💐🙀🆎❌🐻📁🌠🌊🎴🙀🌊🈁🔎🐻🎴💑🈸🌱🈸📻🐻🎴👪🆗🔨🍆🎴🎑💐🎴👪🆗🔨🍆💐🎴🌙💐📻👪💐🈺🎴🎑🌊🙀🍐💐🙀🆎❌🐻📁💐🈺🎴🙀🌊🈁🔎🈺🎴🎑🌊🙀🍐💐🙀🆎❌🐻📁💐🎴💑🈸🙀📁🈸🌊🈁🔨🎴😅🆎🈸🎴🌙🎑🐻🍆💐🎴🈸🎴📁🌊🍐🎑🌠🌱🎴📩🌊🎑🎑🈸🙀🆎🈺🌱🎴🈸🎴❌🆖👉📁🐻🎴🈸🎴📁🌊🆗🌊🎴🈸🎴🙀🌊🈁🔎🌊🎴🈸🎴🍐🌊❌🈸📁🌊🍐🌊🙀🌙🈸🈁🎴👪🈸🙀🆎🈸🐻🎑🙀🆎📁🌊🎴❌🌊📁🎴🆎🈸❌🙀🆎🈸🌟🆗🈸🈁💐🎑🎴🔴🐻🆗🈸🌙🈸🍆🈸🎴👪🌠🌱💐🎑🐻🈺🎴🍐💐🙀🆎🔎🎴💑🌊🆗📁💐🈺🎴❌🌊📁🎴🆎🈸❌🙀🆎🈸🌟🎴💐🎑🎑💐🎴🌙💐🆗🌊🎑🐻🎑💐🎴🆗🈸🈁💐🎑🎴🔴🐻🆗🈸🌙🈸🍆🈸🎴👪🌠🌱💐🎑🐻🈺🎴💐🎑🎑💐🎴🌙💐🆗🌊🎑🐻🎑💐🎴💑🈸🆗💐😨🐻❌💐🎴🙀🈸📁🆗🌊🈁🌊🎑🎑🐻🌙🈸📁🎴📁🙀🌊👪🎑🌊📁🎑🈸🙀🆎🔎🆖🎴🙀🈸👪🌊🆗📻💐🎑🐻🈺🎴🎑🌊🈸👉🌠🍐💐🌟🎑💐🈺🎴🙀📁🈸👉🈸👪💐🎴🆗💐🙀🌙🈸📁💐🎑🎑🈸🙀🆎🔎🎴💑🈸📁🌊🙀🆎📁🈸📁💐🎑🐻🈺🎴🔨👪🐻📁🐻🆎🌊❌🔎🎑🈸🎴🙀🈸🍐🌊🆎💐❌🐻🙀🔎🎴📁🎴😅🆎🈸🈁🎴🆗🈸🈁💐🎑🌊🎴🙀🎴📩🌊❌🔎🎑🈸🙀🆎🔎🆖🎴🌱🔨👪🈸📻🌊🙀🆎📁🌊🎑🎑🈸🍆🈸🎴📁😨🍆❌🈺👪💐🎴💐📁🆎🈸🆗💐🎴🎑💐🎴📻🐻😨🎑🔎🎴🈸🎑🎴📁🌠🙀🆎🔨💑💐❌🎴😨👪🌊🙀🔎🎴🌙💐🌙🎴🌱🔨👪🈸📻🎑🐻🌙🎴🐻🎴🈁🌠🙀❌🐻🆎🌊❌🔎🎴🐻🎴🎑💐😨🎑💐🍐🌊🎑🐻🌊🎴🐻🙀🌙🔨🙀🙀🆎📁💐🎴📁🐻👪🌊❌🎴🎑🌊🎴📁🎴🆎🈸🈁🎴🍐🆎🈸👉🌠🎴🎑🌊🈸🙀💑🈸🆗🐻🈁🈸🎴🆗💐😨🆗🌊🔴🐻🆎🔎🎴📁🈸💑🆗🈸🙀🎴💐🎴📁🎴🆎🈸🈁🎴🍐🆎🈸👉🌠🎴😨💐🙀🆎💐📁🐻🆎🔎🎴❌🆖👉🐻🆎🔎🎴📻🐻😨🎑🔎🎴📁🎴👉🌊🙀🍐🐻🙀❌🌊🎑🎑🌠🌱🎴🎑🐻🌙🈸🍆👪💐🎴🎑🌊🎴🐻🙀🆎🈸🚤🐻🈁🌠🌱🎴📁🙀🌊🌱🎴🌊🌊🎴💑🆗🈸🈺📁❌🌊🎑🐻🈺🌱🎴📁🎴🌊🎴🍆🈸👪🌠🎴🈸👪🐻🎑🎴🈁💐🙀🆎🐻🆎🌠🌟🎴💑🐻🙀💐🆎🌊❌🔎🎴💑🈸📁🐻👪🐻🈁🈸🈁🔨🎴🍆🈸🎑🍐💐🆗🈸📁🎴🙀🌙💐😨💐❌🎴👪🈸🙀🆎🈸🌊📁🙀🌙🈸🈁🔨🎴😅🆎🈸🎴📁🌊🚤🔎🎴🎑🌊🙀❌🌠🌱💐🎑🎑💐🈺🎴😅🆎🈸🎴📁🌊🚤🔎🎴💑🌊🆗📁💐🈺🎴🌙🆎🈸🎴🔨🎴🎑💐🙀🎴🐻😨🎴💑🐻🙀💐🆎🌊❌🌊🌟🎴🈁🈸📻🌊🆎🎴💑🈸🆗💐📁🎑🈺🆎🔎🙀🈺🎴🙀🎴😅🆎🐻🈁🎴💐🎴📁🎴🌊📁🆗🈸💑🌊🎴🌙🆎🈸🎴💑🆗🌊👪🙀🆎💐📁🐻🆎🎴🌱🈸🆎🔎🎴🍐🆎🈸🎑🐻👉🔨👪🔎🎴💑🈸👪🈸👉🎑🈸🌊🎴🔰🈁🎴👪🈸🙀🆎🈸🌊📁🙀🌙🐻🌟🎴🎑💐🌱🈸👪🐻❌🎴

In [71]:
for serach_res in decode_text(encoded_text, bigram_stat, n_gram, beam_search_width=10):
    print(serach_res.phrase)

ыле нылатененуазъенеуюна вщиыхдоваи омойе заымзащео р ыл вшия фидичай лкздканапрь ятгрербаязи фириенвозаеммицеь щаодцуудл  пцубе нплбоя елли сариеасязалрынегли фиришеютбетефля ь мофркуу море бщиихналифивы оодстсь оерера мозаему моъю среъю пдирабрзаыйфиреодасодугфигрхои атжехокощеодщивага к счиюткршия дарозаымзай вае  нскнаыле хои атжехокощеодщивага пале у карай ютздятние з  калнаракорореа елглбрчию знноискаганелебыы  с сязтецабоыхи пошаалбрчиюте глтие вагарьыле руаллиязлывшльятгороь диыхдовавосьэталлифиьнпьла вглтиналищеерязимбо пиззди л  пкнег остчнфимельогнолсон усьла врайсмоазры втагомил нер начуье полфишеер нтао ворозапрк биы елерцеднфаыхяткожечиерщухосье розапрк биы хафиди вгоусжнчиупкнегержуглрицынаетъерако бе  вьефианыещиербеъемяютхо сдаравыеровсаныи нылато си всй пенурумаюте  рледипощебы оо я у хадоа фиазф посяотзаерергоелцещивае я ь морех юткршия ывмопол сьенютентртааяомти сютзд иднчеия с тдупре ромахонионзаюткапую и виь чиотзараалугютна сливоторабрзаыс п гн  внызверо и диыхдо

In [72]:
original_processed

'анна каренина один из самых знаменитых романов льва толстого начинается ставшей афоризмом фразой все счастливые семьи похожи друг на друга каждая несчастливая семья несчастлива посвоему это книга о вечных ценностях о любви о вере о семье о человеческом достоинстве лев толстойроман широкого дыхания часть первая лев толстой анна каренина роман широкого дыхания анна каренина поразила современников вседневностью содержания необычайная свобода раскованность повествования удивительно сочетались в этом романе с цельностью художественного взгляда автора на жизнь он выступал здесь как художник и мыслитель и назначение искусства видел не в том чтобы неоспоримо разрешить вопрос а в том чтобы заставить любить жизнь в бесчисленных никогда не истощимых всех ее проявлениях в е годы один маститый писатель повидимому гончаров сказал достоевскому это вещь неслыханная это вещь первая кто у нас из писателей может поравняться с этим а в европе кто представит хоть чтонибудь подобное фм достоевский находил 